# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words thanks to vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which is the ÌMDB dataset: it is comprised of sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


2022-09-22 20:11:45.082448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 20:11:45.264385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-22 20:11:45.264430: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-22 20:11:45.268433: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown here: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with word2vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with word2vec)
- the representation learned by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As in this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. The third line stores the words and their trained embeddings in `wv`.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [3]:
# YOUR CODE HERE
wv['cat']

array([-0.1220691 ,  0.1864392 , -0.24826041,  0.34586456, -0.03931043,
       -0.32793215,  0.08745287,  0.7517482 , -0.24583463, -0.23926648,
        0.02311643, -0.37340438,  0.09481872,  0.20562841, -0.02056931,
       -0.30328217,  0.25574118, -0.19737384, -0.03342708, -0.5247679 ,
        0.20556715,  0.05338379,  0.32465342, -0.24498837, -0.05605843,
        0.07061789, -0.33383656, -0.08008449, -0.2804675 ,  0.02456995,
        0.20301387, -0.0057568 ,  0.14564975, -0.520303  , -0.26665097,
        0.18895833,  0.1401752 , -0.13151777, -0.31172863, -0.26482525,
       -0.1555909 , -0.38698813, -0.40896332,  0.26221946,  0.36849412,
       -0.00415708, -0.1571079 , -0.13714242,  0.35415888,  0.19907488,
        0.10309789, -0.29473287, -0.2511166 , -0.00374376, -0.2546464 ,
        0.17766011,  0.18731721, -0.06979153, -0.11740136,  0.01578661,
        0.20424591, -0.07291843,  0.0443667 ,  0.18103206, -0.09063219,
        0.25114694, -0.03704687,  0.34360543, -0.44140175,  0.31

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [4]:
# YOUR CODE HERE
len(wv['dog'])

100

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the [`Word2Vec.wv.most_similar`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words that have similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [5]:
# YOUR CODE HERE
word2vec.wv.most_similar('cat', topn=20)

[('door', 0.9862626791000366),
 ('calls', 0.985936164855957),
 ('ann', 0.985741913318634),
 ('stabbing', 0.9839553236961365),
 ('echo', 0.9831508994102478),
 ('marriage', 0.9825789332389832),
 ('ancient', 0.9822781682014465),
 ('boss', 0.981964647769928),
 ('widmark', 0.9817947745323181),
 ('arthur', 0.9815680980682373),
 ('meanwhile', 0.9815651178359985),
 ('slave', 0.9809535145759583),
 ('playboy', 0.9808986186981201),
 ('horse', 0.9808488488197327),
 ('obligatory', 0.9803341627120972),
 ('producer', 0.9799721240997314),
 ('dr', 0.9797556400299072),
 ('built', 0.9793866276741028),
 ('forest', 0.979304313659668),
 ('hank', 0.9792453646659851)]

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [6]:
# YOUR CODE HERE
word2vec.wv.similar_by_vector('cat')

[('door', 0.9862626791000366),
 ('calls', 0.985936164855957),
 ('ann', 0.985741913318634),
 ('stabbing', 0.9839553236961365),
 ('echo', 0.9831508994102478),
 ('marriage', 0.9825789332389832),
 ('ancient', 0.9822781682014465),
 ('boss', 0.981964647769928),
 ('widmark', 0.9817947745323181),
 ('arthur', 0.9815680980682373)]




# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [7]:
# YOUR CODE HERE
wv['good'] - wv['bad']

array([-0.3041764 , -0.42927092,  0.1470629 ,  0.32962584,  0.05376071,
       -0.10096526, -0.3058883 , -0.18594395, -0.03720352, -0.16558933,
        0.17404318,  0.00577629, -0.1244494 ,  0.10811669, -0.21120062,
        0.48912802, -0.41720062,  0.3564334 , -0.28590184,  0.15239978,
       -0.3142368 , -0.07691635, -0.15156841,  0.18587385,  0.03079191,
       -0.0420773 , -0.10343581,  0.6703903 , -0.08444878, -0.69544345,
       -0.26214856,  0.35394222,  0.12946993,  0.32773855, -0.11386472,
       -0.6146598 ,  0.16568516, -0.24723822, -0.5490724 ,  0.82120883,
       -0.1465359 , -0.26265305, -0.40089345, -0.18988371,  0.57754   ,
       -0.17052744,  0.5301707 , -0.16377085, -0.10196328,  0.50974333,
       -0.05007438, -0.17318183, -0.54426336,  0.26519257,  0.01759636,
        0.12917125, -0.23539823,  0.15821892, -0.00313985,  0.5733511 ,
        0.05188437,  0.21645182, -0.09603888, -0.45526797,  0.10030282,
        0.0065999 , -0.49699205, -0.16102989,  0.3920136 ,  0.51

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [8]:
# YOUR CODE HERE
res = wv['good'] - wv['bad'] + wv['stupid']
res

array([-0.14742303, -0.46328047,  0.28256118, -0.08036128,  0.15885128,
       -0.79601526, -0.6142702 ,  0.2975976 , -0.4326711 , -0.44723946,
       -0.02126257, -0.3493589 , -0.01740228,  0.357698  ,  0.02196117,
        0.2401931 , -0.10112688, -0.30160075, -0.43052012, -0.66977125,
        0.1697506 ,  0.0642605 ,  0.41727412,  0.07297279, -0.12808101,
       -0.17115574, -0.4216179 ,  0.5870405 , -0.2749799 , -0.6560691 ,
        0.39336795,  0.4100685 ,  0.41582954, -0.07189634, -0.4206413 ,
        0.20534205,  0.12375932, -0.08344187, -0.7442714 , -0.07818925,
       -0.17336568, -0.7869979 , -0.21674302,  0.32546645,  0.8113414 ,
       -0.07638479,  0.11275598, -0.49095407, -0.20562464,  0.581043  ,
        0.17977738, -0.32032865, -0.6080267 ,  0.38513994,  0.0971515 ,
        0.4804707 , -0.03069665,  0.14672026, -0.45268968,  0.17707667,
        0.04031291, -0.00619586,  0.27323347, -0.06219831, -0.5145888 ,
        0.49359223, -0.3183387 ,  0.03689015,  0.26506442,  0.70

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `similar_by_vector` function) of `res`

In [9]:
# YOUR CODE HERE
word2vec.wv.similar_by_vector(res)

[('nice', 0.7980955243110657),
 ('good', 0.7397701144218445),
 ('decent', 0.7324145436286926),
 ('great', 0.7225892543792725),
 ('given', 0.7225058674812317),
 ('developed', 0.7093029618263245),
 ('always', 0.708748459815979),
 ('rotten', 0.7081645727157593),
 ('surprises', 0.7000495195388794),
 ('smart', 0.6992700695991516)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [10]:
# YOUR CODE HERE
test = wv['boy'] - wv['girl'] + wv['woman']

In [11]:
wv['man'] - test

array([-0.37901765, -0.1712875 ,  0.09056823, -0.06326824,  0.15760794,
       -0.02242076, -0.3084498 ,  0.35362363, -0.18532088, -0.45733643,
       -0.5246047 , -0.13058877, -0.25143403, -0.21914075,  0.00900382,
       -0.2029623 ,  0.02978848, -0.1963307 , -0.04149136,  0.04035068,
       -0.04903742,  0.2715497 , -0.11465123,  0.01439419,  0.05427976,
       -0.06340916, -0.4168918 ,  0.487152  , -0.02803257, -0.02213357,
       -0.04387899,  0.02582806,  0.0348956 , -0.39287388,  0.19738813,
        0.14570372,  0.17070848,  0.02327561,  0.21389219,  0.44716907,
        0.08290406, -0.30374426, -0.3026169 ,  0.52039886,  0.04227006,
       -0.26450983,  0.3366387 , -0.14077511,  0.37281877, -0.19006106,
        0.46696198, -0.00687069,  0.18890306,  0.19165246,  0.34227192,
        0.5761306 , -0.03881878, -0.05149671,  0.04638916,  0.20485738,
        0.02988911,  0.4834277 ,  0.32311368, -0.15394238,  0.2612468 ,
       -0.21320783,  0.39110556,  0.37223887,  0.3770005 , -0.04

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's check some out. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [12]:
# YOUR CODE HERE
word2vec2 = Word2Vec(sentences=X_train, vector_size=50)
wv2 = word2vec2.wv

❓ **Question** ❓ Use the `Word2Vec.wv.key_to_index` attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [13]:
# YOUR CODE HERE
len(wv.key_to_index), len(wv2.key_to_index)

(8006, 8006)

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [14]:
# YOUR CODE HERE
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=5)
wv3 = word2vec_3.wv

word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=2)
wv4 = word2vec_4.wv

In [15]:
len(wv4.key_to_index), len(wv3.key_to_index)

(16729, 8006)

Remember that we said word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [16]:
# YOUR CODE HERE
word2vec_5 = Word2Vec(sentences=X_train, vector_size=50, min_count=10, window=10)
wv5 = word2vec_5.wv
wv5['dog']

array([ 1.0080025e-01,  1.6887759e-01,  1.2031399e-01,  1.7130253e-01,
       -1.1206067e-02,  1.2383969e-04,  1.4050254e-01,  7.0515925e-01,
       -6.0777968e-01,  1.7554913e-02, -4.9434647e-01, -1.0199873e-01,
        5.5636972e-02,  6.5622613e-02,  6.0003746e-04,  2.0039099e-01,
        1.7637344e-01,  3.5761100e-01, -3.4364042e-01, -4.3111062e-01,
        2.8006575e-01, -2.4063904e-03,  5.1056916e-01, -3.2442424e-01,
        3.0470195e-01,  3.7481165e-01, -1.2334785e-01,  1.9484766e-01,
       -4.5005506e-01,  7.3936738e-02, -4.5829859e-01, -1.2872449e-01,
       -2.2514899e-01,  3.9166024e-01, -1.3572088e-01,  3.3400553e-01,
        3.0540559e-01,  5.7857488e-03,  3.9992318e-01, -3.3053660e-01,
        4.3783437e-02, -4.8278969e-02, -1.9942312e-01,  2.1877041e-02,
        6.3732725e-01,  1.3439064e-01,  7.9883441e-02, -4.2432809e-01,
        4.0896934e-01,  3.4267530e-01], dtype=float32)

In [17]:
example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_test = [text_to_word_sequence(_) for _ in example]
[wv5[x] for x in example_test]

[array([[-0.87487185,  0.43133703, -1.099333  , -1.6261605 , -1.2104224 ,
         -1.1671977 ,  1.9988258 ,  1.1578234 , -1.1142917 , -0.76286024,
          0.3619942 ,  0.06722049, -1.8714229 , -0.7684049 , -1.6192098 ,
         -1.6528227 , -0.2889923 ,  0.28725687, -1.8295366 , -0.5650632 ,
         -0.56300783,  2.3834822 ,  0.50825495,  2.320632  , -1.432137  ,
          0.14022477,  0.54905015, -1.4295777 , -0.37952363,  2.1012163 ,
          1.3563061 , -1.2576945 ,  1.7123203 , -1.724448  , -0.5763391 ,
          0.6125608 ,  1.7086902 ,  0.01861261,  0.63739973, -1.2736809 ,
          0.62874043, -1.7800695 , -1.9071778 , -0.3009988 , -0.826817  ,
         -1.9650267 ,  0.7873064 ,  1.3325982 , -0.3330205 ,  0.28348014]],
       dtype=float32),
 array([[-1.4663258 , -0.8308287 , -1.3002037 , -2.6924412 , -0.04084561,
         -1.1553656 ,  1.1419599 ,  0.97364736,  0.9971756 , -1.0452819 ,
          1.2240272 , -0.56186724,  2.222639  ,  0.14556862, -2.2166064 ,
          0.8

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [ ]:
word2vec()

In [44]:
import numpy as np
from tensorflow.keras.preprocessing.text import text_to_word_sequence

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']


def embed_sentence(word2vec, sentence):
    matrix = []
    for word in sentence:
        if word in word2vec.wv:
            matrix.append(word2vec.wv[word])
    
    return np.array(matrix)

### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))


In [45]:
embedded_sentence.shape

(8, 100)

In [46]:
embedded_sentence_missing_words.shape

(3, 100)

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [47]:
def embedding(word2vec, sentences):
    lista = []
    for sentence in sentences:
        lista.append(embed_sentence(word2vec, sentence))
    return lista
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

In [51]:
X_train

[array([[ 0.74382997,  0.1532751 ,  0.83061874, ..., -0.62375206,
          0.15864526, -1.4629138 ],
        [ 0.3369675 , -0.45619112,  0.88788116, ..., -1.7825085 ,
          1.1539309 ,  0.28301764],
        [ 0.5366162 , -0.01002333, -0.11539467, ..., -0.84922487,
         -0.4177939 ,  0.36553228],
        ...,
        [-0.14336894, -0.06834376,  0.00232658, ..., -0.3178301 ,
          0.10496569, -0.02943785],
        [-0.2961204 , -0.00680413, -0.34228534, ..., -0.34798118,
          0.3971282 ,  0.06019743],
        [ 0.01539697,  0.06569556,  0.13545561, ..., -0.2884566 ,
         -0.3243955 , -0.6398351 ]], dtype=float32),
 array([[-0.47778785,  1.1442813 ,  1.4320815 , ...,  0.36239833,
          0.4718153 , -1.1002933 ],
        [-1.542467  , -0.21702465,  0.10522654, ..., -1.0625558 ,
         -0.39956567, -0.76397955],
        [-1.2801539 , -0.02574848,  1.1470991 , ..., -0.6420514 ,
         -0.5409684 , -0.12360486],
        ...,
        [ 0.80204725, -0.04394266, -0.0

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors that can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [53]:
### YOUR CODE HERE
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)